## Deep Research

In [2]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

# pushover_user = os.getenv("PUSHOVER_USER")
# pushover_token = os.getenv("PUSHOVER_TOKEN")
# pushover_url = "https://api.pushover.net/1/messages.json"

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

### 1. Search Agent

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [4]:
message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of May 2025, several AI agent frameworks have gained prominence for their capabilities and developer adoption:

- **LangChain**: A modular Python framework enabling the creation of LLM-powered agents with features like prompt chaining, memory integration, and external tool utilization. It supports various LLMs, including OpenAI and Cohere, and offers a rich ecosystem of tools for tasks such as web search and code execution. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen facilitates the development of multi-agent systems through natural language communication between agents. It includes built-in agents like AssistantAgent and UserProxyAgent, and integrates with OpenAI and Azure OpenAI endpoints. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **CrewAI**: This framework focuses on orchestrating role-based AI agents for collaborative tasks, emphasizing simplicity and minimal setup. It has gained significant popularity, with over 32,000 GitHub stars and nearly 1 million monthly downloads, particularly in customer service and marketing automation. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, designed for creating multi-agent workflows with comprehensive tracing and guardrails. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **LangGraph**: An extension of LangChain, LangGraph allows developers to define agents as state machines, providing fine-grained control over agent state and facilitating complex workflows. It integrates with LangChain’s tools, memory, and models, supporting loops, conditional branching, and persistent state. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

These frameworks are at the forefront of AI agent development, offering diverse approaches to building intelligent, autonomous systems. 

### 2. Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [6]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan,
)

In [7]:

message = "What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='Identify the leading AI agent frameworks currently known for popularity and success.', query='top AI agent frameworks 2025'), WebSearchItem(reason='Gather expert reviews and comparisons of AI agent frameworks in 2025.', query='best AI agent frameworks comparison May 2025'), WebSearchItem(reason='Find recent trends and market share of AI agent frameworks in early 2025.', query='AI agent frameworks market share 2025'), WebSearchItem(reason='Look for user and developer feedback on popular AI agent frameworks in 2025.', query='user reviews AI agent frameworks 2025'), WebSearchItem(reason='Discover technological advancements and new features in AI agent frameworks as of May 2025.', query='latest features AI agent frameworks 2025')])


### 3. Writer Agent

Take the results of internet searches and make a report

In [8]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

### 4. push notification


In [ ]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [ ]:
push

In [9]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required")
)

NameError: name 'push' is not defined

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [10]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### send a push notification

In [12]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

## The show

In [14]:
query ="What are the most popular and successful AI Agent frameworks in May 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    # await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Hooray!


# Research Report on AI Agent Frameworks in May 2025

## Overview
The rapid evolution and integration of Artificial Intelligence (AI) in business processes have brought forth numerous frameworks designed for developing AI agents. By May 2025, several of these frameworks have distinguished themselves through performance, popularity, and versatility. This report delves into the leading AI agent frameworks, their unique functionalities, and case studies illustrating their applications across industries.

## Outline of Report
1. **Introduction**  
   1.1 Definition of AI Agents  
   1.2 Importance of AI Agent Frameworks  

2. **Leading AI Agent Frameworks**  
   2.1 LangChain  
   2.2 AutoGen  
   2.3 CrewAI  
   2.4 OpenAI Agents SDK  
   2.5 Google Agent Development Kit (ADK)  
   2.6 Other Notable Frameworks  

3. **Industry Applications of AI Agents**  
   3.1 Financial Services  
   3.2 Healthcare  
   3.3 Software Development  
   3.4 Retail  

4. **Market Trends and Projections**  
   4.1 Growth Statistics and Predictions  
   4.2 Adoption Rates Across Industries  
   4.3 Challenges and Concerns  

5. **Conclusion**  
   5.1 Summary of Findings  
   5.2 Future Directions  

6. **References**  

## 1. Introduction
### 1.1 Definition of AI Agents
AI agents are autonomous or semi-autonomous software entities that perform tasks on behalf of users. They can interact with other agents, understand user commands, and manage workflows to execute complex processes efficiently.
### 1.2 Importance of AI Agent Frameworks
AI agent frameworks provide the underlying structures needed to build, manage, and deploy these agents, making them essential for organizations aiming to leverage AI for process automation and operational efficiency.

## 2. Leading AI Agent Frameworks
### 2.1 LangChain
LangChain is a modular Python framework aimed at building applications with large language models (LLMs). Its key features include:
- **Prompt Chaining**: Enables the sequential use of multiple prompts.
- **Tool Integration**: Facilitates the incorporation of APIs and other tools.
- **Memory Management**: Allows for the retention of information between interactions.

As of May 2025, LangChain boasts over 70,000 stars on GitHub, showcasing its popularity among developers.

### 2.2 AutoGen
Developed by Microsoft, AutoGen focuses on creating multi-agent systems that communicate through natural language. Key attributes include:
- **Conversational Coordination**: Agents can interact in a conversation-like manner to collaborate.
- **User-Friendly Interface**: Provides tools for prototyping and streamlining agent development.

### 2.3 CrewAI
CrewAI emphasizes collaborative task execution among AI agents. Major features include:
- **Role-Based Task Execution**: Each agent can take on specific roles, simplifying collaborative efforts.
- **Quick Deployment**: Minimal setup allows for faster implementation in various sectors.

### 2.4 OpenAI Agents SDK
The OpenAI Agents SDK is a lightweight framework tailored for multi-agent workflows. Features include:
- **Tracing and Guardrails**: Comprehensive tracking of interactions enhances security and debugging.
- **Compatibility**: Supports over 100 different LLMs for flexibility in agent design.

### 2.5 Google Agent Development Kit (ADK)
Emerging in April 2025, the Google ADK integrates seamlessly with Google’s ecosystem. Features include:
- **Hierarchical Agent Composition**: Simplifies complex tasks by allowing agents to be arranged in hierarchy.
- **Minimal Code Requirement**: Facilitates faster development cycles and reduces complexity.

### 2.6 Other Notable Frameworks
Other noteworthy frameworks include LangGraph, which focuses on stateful, graph-based agent workflows, and Semantic Kernel, designed for enterprise-grade workflows with a strong focus on security.

## 3. Industry Applications of AI Agents
### 3.1 Financial Services
In the financial sector, a global firm implemented an AI agent system to streamline loan approvals, reducing processing time and cutting operational costs significantly.  
### 3.2 Healthcare
A large hospital network utilized AI agents for scheduling and insurance verification systematization, which helped decrease the administrative load by 30%.
### 3.3 Software Development
In technology, firms like Microsoft integrated GitHub's Copilot, allowing AI agents to assist in coding and documentation, leading to a 35% surge in development velocity.
### 3.4 Retail
Retail companies adopted AI agents to provide personalized shopping assistance, resulting in improved conversion rates and decreased customer support queries.

## 4. Market Trends and Projections
### 4.1 Growth Statistics and Predictions
The global AI agent market is projected to increase from $7.84 billion in 2025 to an astounding $236 billion by 2034, representing a CAGR of 45.8%.  
### 4.2 Adoption Rates Across Industries
An estimated 85% of enterprises plan to integrate AI agents into their operations by 2025, indicating a shift towards AI-led automation.
### 4.3 Challenges and Concerns
Challenges including cybersecurity, lack of standardization, and potential data breaches must be addressed to fully realize the benefits of AI agents.

## 5. Conclusion
### 5.1 Summary of Findings
The leading AI agent frameworks as of May 2025 offer diverse functionalities enabling significant automation and efficiency improvements across various domains.
### 5.2 Future Directions
As the AI agent landscape evolves, ongoing developments will likely focus on enhancing security protocols, expanding integration capabilities, and refining user interfaces for broader accessibility.

## 6. References
- [ODSC](https://odsc.medium.com/top-10-open-source-ai-agent-frameworks-to-know-in-2025-c739854ec859?utm_source=openai)
- [Medium](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai)
- [DataCamp](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai)
- [Axios](https://www.axios.com/2025/05/06/ai-agents-identity-security-cyber-threats?utm_source=openai)  
- [TechJury](https://techjury.net/industry-analysis/ai-agent-market-size-share/?utm_source=openai)  

**Prepared by:** Senior Researcher, AI & Automation Division  
**Date:** October 2023  

---  
This report serves as a detailed overview of the prominent AI agent frameworks as of May 2025, showcasing their capabilities and industry utilization, while also considering market growth and future trends.

### As always, take a look at the trace

https://platform.openai.com/traces